## AI-HUB 데이터 확인

In [ ]:
import os
from glob import glob

# 1. 모든 json 파일 경로 수집 (재귀적으로)
json_paths = glob('./01_data_meta/**/*.json', recursive=True)

# 2. 파일 이름만 추출 (중복 제거용)
json_filenames = set(os.path.basename(path) for path in json_paths)

# 3. 개수 출력
print(f"📄 총 JSON 파일 수: {len(json_paths)}")
print(f"🆔 유니크한 JSON 파일 이름 수: {len(json_filenames)}")


### 메타 정보 csv 생성

In [ ]:
import json
from glob import glob
import os
import pandas as pd

# JSON 파일 경로 수집
json_paths = glob("01_data_meta/**/*.json", recursive=True)

records = []

for path in json_paths:
    with open(path, 'r', encoding='utf-8') as f:
        data = json.load(f)
        
        # 안전한 접근을 위한 get
        meta = data.get('metadata', {})
        animal = meta.get('animal', {})
        owner = meta.get('owner', {})
        inspect = meta.get('inspect', {})

        records.append({
            "json_file": os.path.basename(path),
            "breed": animal.get("breed"),
            "cat_action": inspect.get("action"),
            "cat_emotion": inspect.get("emotion"),
            "owner_situation": owner.get("situation"),
            "owner_emotion": owner.get("emotion"),
            "seq": meta.get("seq"),
            "freams_anotations" : len(data['annotations']),
        })

# DataFrame 변환
df = pd.DataFrame(records)

# 미리보기
print(df.head())

# 필요 시 저장
# df.to_csv("./01_data_meta/all_cat_metadata.csv", index=False)


In [ ]:
# 혹시 몰라서 추가 확인 작업
# 서로 다른 값이 있는 행만 필터링
diff_df = df[df['seq'] != df['freams_anotations']]

print(f"seq와 freams_anotations 값이 다른 경우: {len(diff_df)}건")
print(diff_df[['json_file', 'seq', 'freams_anotations']])


### 데이터 전처리(프레임 기준 필요없는 동영상_json 제거하기)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# CSV 파일 로드
df = pd.read_csv("./01_data_meta/all_cat_metadata.csv")

# 기본 통계 정보 출력
print("📊 프레임 수 통계 요약:")
print(df["freams_anotations"].describe())

# 히스토그램 시각화
plt.figure(figsize=(10, 6))
sns.histplot(df["freams_anotations"], bins=30, kde=True, color='skyblue')
plt.title("Distribution of Frame Counts per Video")
plt.xlabel("Number of Frames")
plt.ylabel("Number of Videos")
plt.grid(True)
plt.show()
